$\newcommand{\mb}[1]{\mathbf{ #1 }}$
$\newcommand{\bs}[1]{\boldsymbol{ #1 }}$
$\newcommand{\bb}[1]{\mathbb{ #1 }}$

$\newcommand{\R}{\bb{R}}$

$\newcommand{\ip}[2]{\left\langle #1, #2 \right\rangle}$
$\newcommand{\norm}[1]{\left\Vert #1 \right\Vert}$

$\newcommand{\der}[2]{\frac{\mathrm{d} #1 }{\mathrm{d} #2 }}$
$\newcommand{\derp}[2]{\frac{\partial #1 }{\partial #2 }}$

# Finite Dimensional Koopman Bilinear System

Consider a nonlinear dynamical system that allows an exact finite dimensional Koopman canonical transform such that the control-affine dynamics can be transformed to a bilinear system. Consider the dynamical system 

\begin{equation}
\mb{\dot{x}}=\mb{f}_0(\mb x) + \mb f_1 ( \mb x) u_1 + \mb f_2(\mb x) u_2, 
\end{equation}

where we for this example choose $\mb f_0, \mb f_1$ as follows:

\begin{equation}
\mb f_0(\mb x) = \begin{bmatrix} x_3 \\ x_4 \\ \lambda x_3 \\ \mu x_4 + (2 \lambda - \mu) c x_3^2 \end{bmatrix}, \qquad
\mb f_1(\mb x) = \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix}, \qquad
\mb f_2(\mb x) = \begin{bmatrix} 0 \\ 0 \\ 0 \\ x_1+1 \end{bmatrix}, 
\end{equation}   

and $\lambda, \mu, c \in \mathbb{R}$ are scalar parameters of the system. Setting $ \mb x = [q_1 \, q_2 \, \dot{q_1} \, \dot{q_2}]^T$, 
these equations of motion can be expressed as robotic dynamics of the form $\mb{D}(\mb{q})\ddot{\mb{q}} + \mb{C}(\mb{q}, \dot{\mb{q}})\dot{\mb{q}} + \mb{G}(\mb{q}) = \mb{B}\mb{u}$, where $\mb D$ is the inertia matrix, $\mb C$ is the matrix of Coriolis terms, $\mb G$ is the matrix of gravitational terms, and $\mb B$ is the static actuation matrix. Rewriting $\mb f_0, \mb f_1, \mb f_2$ in terms of $\mb D, \mb C, \mb G,$ and $\mb B$ yield


\begin{equation}
\mb D(\mb q) = \begin{bmatrix} 1 & 0\\ 0 &  \frac{1}{q_1+1} \end{bmatrix}, 
\qquad \mb C(\mb q, \mb{\dot{q}}) = -\begin{bmatrix} \lambda & 0 \\ \frac{1}{q_1 + 1}(2 \lambda - \mu) c \dot{q}_1 & \frac{1}{q_1 +1} \mu \end{bmatrix}, \qquad 
\mb G(\mb q) = \begin{bmatrix} 0 \\ 0 \end{bmatrix} 
\qquad \mb B = \begin{bmatrix}1 & 0 \\ 0 & 1 \end{bmatrix},
\end{equation}

In [23]:
import numpy as np

from core.dynamics import RoboticDynamics

class FiniteDimKoopSys(RoboticDynamics):
    def __init__(self, lambd, mu, c):
        RoboticDynamics.__init__(self, 2, 2)
        self.params = lambd, mu, c
    
    def D(self, q):
        return np.array([[1, 0],[0, (q[0]+1)**(-1)]])
    
    def C(self, q, q_dot):
        labmd, mu, c = self.params
        return -np.array([[lambd, 0], [(q[0]+1)**(-1)*(2*lambd - mu)*c*q_dot[0], (q[0]+1)**(-1)*mu]])
    
    def G(self, q):
        return np.array([0, 0])
    
    def B(self, q):
        return np.array([[1, 0], [0, 1]])
    
lambd, mu, c = .3, .2, -.5
finite_dim_koop_sys = FiniteDimKoopSys(lambd, mu, c)

### Koopman Canonical Transform of the Control-Affine Dynamics

As a result of the careful construction of this system, there exists a Koopman canonical transform, $\mb z = T(\mb x)$ that exactly transforms the control-affine dynamics into a bilinear system. Consider the transformation:

\begin{equation}
    T(\mb q, \mb{\dot{q}}) = \begin{bmatrix} 
    \phi_1(\mb q, \mb{\dot{q}})\\
    \phi_2(\mb q, \mb{\dot{q}})\\
    \phi_3(\mb q, \mb{\dot{q}})\\
    \phi_4(\mb q, \mb{\dot{q}})\\
    \phi_5(\mb q, \mb{\dot{q}})\\
    \phi_6(\mb q, \mb{\dot{q}})\\
    \end{bmatrix}
    = \begin{bmatrix} 
    q_1 - \frac{1}{\lambda}\dot{q}_1\\
    q_2 - \frac{1}{\mu} \dot{q}_2 + \frac{(2 \lambda - \mu)c}{2\lambda \mu} \dot{q}_1^2\\
    \dot{q}_1\\
    \dot{q}_2 - c \dot{q}_1^2\\
    \dot{q}_1^2\\
    1
    \end{bmatrix},
\end{equation}

where $\phi_1, \phi_2, \phi_3, \phi_4, \phi_5, \phi_6$ are eigenfunctions of the Koopman operator associated with the drift 
vector field $\mb f_0$. The matrix with the eigenvalue associated with the i-th eigenfunction on the i-th diagonal 
element is $F=\text{diag}(0, 0, \lambda, \mu, 2 \lambda, 0)$. Then, to reformulate the dynamics we have:

\begin{equation} 
L_{\mb f_1} T(\mb q, \mb{\dot{q}}) = \begin{bmatrix} -\frac{1}{\lambda}\\ \frac{(2\lambda - \mu)c}{\lambda \mu}\dot{q}_1\\ 1 \\ -2c\dot{q}_1 \\ 2\dot{q_1} \\ 0 \end{bmatrix}, \qquad 
L_{\mb f_2} T(\mb q, \mb{\dot{q}}) = \begin{bmatrix} 0\\ -\frac{1}{\mu}(q_1 + 1)\\0 \\ q_1 + 1 \\ 0 \\ 0 \end{bmatrix} 
\end{equation}

and the dynamics can be equivalently transformed to a bilinear form $\mb{\dot{z}} = F \mb z + G_1\mb z u_1 + G_2\mb z u_2$ with

\begin{equation}
F = \begin{bmatrix} 
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & \lambda & 0 & 0 & 0\\
0 & 0 & 0 & \mu & 0 & 0\\
0 & 0 & 0 & 0 & 2 \lambda & 0\\
0 & 0 & 0 & 0 & 0 & 0\\  
\end{bmatrix}, \qquad 
G_1 = \begin{bmatrix} 
0 & 0 & 0 & 0 & 0 & -\frac{1}{\lambda}\\
0 & 0 & \frac{(2\lambda - \mu)c}{\lambda \mu} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1\\
0 & 0 & -2c & 0 & 0 & 0\\
0 & 0 & 2 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\  
\end{bmatrix}
, \qquad 
G_2 = \begin{bmatrix} 
0 & 0 &  0 & 0 & 0 & 0\\
-\frac{1}{\mu} & 0 &  -\frac{1}{\lambda \mu} & 0 & 0 & -\frac{1}{\mu}\\
0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & \frac{1}{\lambda} & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\  
\end{bmatrix}
\end{equation}

In [24]:
from koopman_core.dynamics.bilinear_dynamics import BilinearDynamics

F = np.array([[0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0],
              [0, 0, lambd, 0, 0, 0],
              [0, 0, 0, mu, 0, 0],
              [0, 0, 0, 0, 2*lambd, 0],
              [0, 0, 0, 0, 0, 0]])
G_1 = np.array([[0, 0, 0, 0, 0, -1/lambd],
                [0, 0, (2*lambd - mu)*c/(lambd*mu), 0, 0, 0],
                [0, 0, 0, 0, 0, 1], 
                [0, 0, -2*c, 0, 0, 0], 
                [0, 0, 2, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0]])
G_2 = np.array([[0, 0, 0, 0, 0, 0],
                [-1/mu, 0, -1/(lambd*mu), 0, 0, -1/mu],
                [0, 0, 0, 0, 0, 0], 
                [1, 0, 1/lambd, 0, 0, 1], 
                [0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0]])
G = [G_1, G_2]
phi_fun = lambda x: np.array([x[0] - 1/lambd*x[2],
                            x[1] - 1/mu*x[3] + (2*lambd - mu)*c/(2*lambd*mu)*x[2]**2,
                            x[2], 
                            x[3] - c*x[2]**2, 
                            x[2]**2, 
                            1])
n, m = 6, 2

koop_bilinear_sys = BilinearDynamics(n, m, F, G, phi_fun)

With the same objective as before, we aim to design a feedback linearizing control law utilizing the bilinear dynamics. 
We assume that the outputs can be linearly described in the lifted state space such that $\mb y = C^{\mb h} \mb z$. 
The outputs encoding the objective (driving the system to the origin) can then be described as

\begin{equation}
    \bs{\eta}(\mb{q}, \dot{\mb{q}}) = C^{\mb h} \mb z - \mb y_d(t), 
    \qquad C^{\mb h} = \begin{bmatrix} 1 & 0 & \frac{1}{\lambda} & 0 & 0 & 0\\
    0 & 1 & 0 & \frac{1}{\mu} & \frac{c}{\mu} - \frac{(2\lambda - \mu)c}{2\lambda \mu} & 0
    \end{bmatrix}
\end{equation}

Differentiating the outputs we get and writing the actuated dynamics $G_1 \mb z u_1 + G_2 \mb z u_2$ as $\hat{G}(\mb z) \mb u$ we get:

\begin{equation}
    \bs{\dot{\eta}} = C^{\mb h}\mb{\dot{z}} - \mb{\dot{y}}_d(t) = C^{\mb h} \big (\mb{\dot{z}} - \mb{\dot{z}}_d(t) \big ) = C^{\mb h} \big ( F \mb z + \hat{G}(\mb z)\mb u - \mb{\dot{z}}_d \big ).
\end{equation}

Because of the relative degree assumption on the outputs, each output $y_i(\mb x)=h_i(\mb x), i=1,\dots,k$ has relative degree 2,
we have that $L_g h(x) = C^{\mb h} \hat{G}(\mb z) = 0$. As a result, $\bs{\dot{\eta}} = C^{\mb h} \big ( F \mb z - \mb{\dot{z}}_d \big )$.
Now, akin to classical feedback linearization we differentiate the outputs again to get:

\begin{equation}
    \bs{\ddot{\eta}} = C^{\mb h} \big ( F \mb{\dot{z}} - \mb{\ddot{z}}_d \big ) = C^{\mb h} \big ( F(F \mb z + \hat{G}(\mb z)\mb u)- \mb{\ddot{z}}_d \big )
    = C^{\mb h} \big ( F^2 \mb z + F\hat{G}(\mb z)\mb u - \mb{\ddot{z}}_d \big )
\end{equation}

Now, the output error dynamics can be written in normal form such that we get:

\begin{equation}
    \bs{\Phi}(\mb{q}, \dot{\mb{q}}) = \begin{bmatrix} \bs{\eta}(\mb{q}, \dot{\mb{q}})\\ \bs{\dot{\eta}}(\mb{q}, \dot{\mb{q}})\end{bmatrix}, \qquad
    \bs{\dot{\Phi}} = \begin{bmatrix} \bs{\dot{\eta}}\\ \bs{\ddot{\eta}}\end{bmatrix} =
    \begin{bmatrix} \bs{\dot{\eta}}\\
    C^{\mb h} \big ( F(F \mb z + \hat{G}(\mb z)\mb u)- \mb{\ddot{z}}_d \big ) \end{bmatrix}
\end{equation}

In [25]:
C_h = np.array([[1, 0, 1/lambd, 0, 0, 0], 
                [0, 1, 0, 1/mu, c/mu - ((2*lambd - mu)*c)/(2*lambd*mu), 0]])


### Collect data from the system under PD control

We first collect data from the system under PD control designed by LQR on the linearization of the dynamics around the 
origin. This is done by picking 20 intial conditions $\mb x_0 = [q_0, \dot{q}_0]^T$ uniformly at random in the space 
$\mb x_0 \in [-2, 2]^4$. The linearized dynamics around the origin are:

In [26]:
from core.controllers import PDController
from scipy.linalg import solve_continuous_are

n_lin = 4
A_lin = np.array([[0, 0, 1, 0],
                  [0, 0, 0, 1],
                  [0, 0, lambd, 0],
                  [0, 0, 0, mu]])
B_lin = np.array([[0, 0],
                  [0, 0],
                  [1, 0],
                  [0, 1]])
lin_dynamics = (A_lin, B_lin)

q, r = 5, 1
Q_fb = q * np.identity(4)
R_fb = r*np.identity(2)
P = solve_continuous_are(A_lin, B_lin, Q_fb, R_fb)
K = np.linalg.inv(R_fb)@B_lin.T@P
K_p, K_d = K[:,:int(n_lin/2)], K[:,int(n_lin/2):]
pd_controller = PDController(finite_dim_koop_sys, K_p, K_d)

Finally, we collect the data and store the state and action pairs that will be used to learn dynamics models later on:

In [27]:
import random as rn
from koopman_core.controllers.perturbed_controller import PerturbedController

n_traj = 50                                                 # Number of trajectories to collect data from
dt = 1.0e-2                                                 # Time step length
N = int(5./dt)                                              # Number of time steps
t_eval = dt * np.arange(N + 1)                              # Simulation time points
traj_bounds = [2., 2., 2., 2.]                                        # State constraints, [q, q_dot]
pert_noise_var = 2.                                        # Variance of controller perturbation noise

pert_lqr_controller = PerturbedController(finite_dim_koop_sys, pd_controller, pert_noise_var)

xs, us, us_nom, ts = [], [], [], []
for ii in range(n_traj):
    x_0 = np.asarray([rn.uniform(-i,i)  for i in traj_bounds])
    xs_tmp, us_tmp = finite_dim_koop_sys.simulate(x_0, pert_lqr_controller, t_eval)
    xs.append(xs_tmp)
    us.append(us_tmp)
    ts.append(t_eval)

xs, us, ts = np.array(xs), np.array(us), np.array(ts)

### Learn lifted linear state space model with EDMD

To learn a lifted state space model with EDMD, we use a monomial basis. I.e. we use all monomials up a specified degree
such that the lifting function $\bs{\phi}(\mb x)$ is

\begin{equation}
\mb z = \bs{\phi}(\mb x) = [1 \quad x_1 \quad \dots  \quad x_n \quad x_1^2 \quad x_1 x_2 \quad x_n^2 \quad \dots ]^T
\end{equation}

Using the EDMD algorithm, sampled vectors of the state and control action pairs $(\mb x, \mb u)$ are collected, the state
is lifted, $\mb z = \bs{\phi}(\mb x)$, and least squares
regression is used to find $A_{\mb z}, B_{\mb z}$ that best fit the data resulting in a model of the form:

\begin{equation}
\dot{\mb{z}} = A_{\mb z}\mb z + B_{\mb z} \mb u
\end{equation}

In [28]:
from core.dynamics.linear_system_dynamics import LinearSystemDynamics
from koopman_core.basis_functions.bilinear_basis_functions import BilinearBasis
from koopman_core.learning import Edmd
from sklearn import preprocessing
from pysindy.optimizers.stlsq import STLSQ

n, degree = 4, 2
threshold_edmd = 7.5e-2
threshold_bedmd = 2e-1

poly_features_edmd = preprocessing.PolynomialFeatures(degree=3)
poly_features_edmd.fit(np.zeros((1,n)))
n_lift_edmd = poly_features_edmd.n_output_features_
C_edmd = np.zeros((n,n_lift_edmd))
C_edmd[:,1:n+1] = np.eye(n)

basis_edmd = lambda x: poly_features_edmd.transform(x)
optimizer_edmd = STLSQ(threshold=threshold_edmd)
standardizer_edmd = preprocessing.StandardScaler(with_mean=False)

model_edmd = Edmd(n, m, basis_edmd, n_lift_edmd, n_traj, optimizer_edmd, standardizer=standardizer_edmd, C=C_edmd)
X_edmd, y_edmd = model_edmd.process(xs, us, ts)
model_edmd.fit(X_edmd, y_edmd)
sys_edmd = LinearSystemDynamics(model_edmd.A, model_edmd.B)

/Users/carlaxelfolkestad/.conda/envs/keedmd/lib/python3.7/site-packages/pysindy/optimizers/stlsq.py:156: UserWarning: Sparsity parameter is too big (0.075) and eliminated all coefficients
  "coefficients".format(self.threshold)
/Users/carlaxelfolkestad/.conda/envs/keedmd/lib/python3.7/site-packages/pysindy/optimizers/stlsq.py:156: UserWarning: Sparsity parameter is too big (0.075) and eliminated all coefficients
  "coefficients".format(self.threshold)
/Users/carlaxelfolkestad/.conda/envs/keedmd/lib/python3.7/site-packages/pysindy/optimizers/stlsq.py:156: UserWarning: Sparsity parameter is too big (0.075) and eliminated all coefficients
  "coefficients".format(self.threshold)
/Users/carlaxelfolkestad/.conda/envs/keedmd/lib/python3.7/site-packages/pysindy/optimizers/stlsq.py:156: UserWarning: Sparsity parameter is too big (0.075) and eliminated all coefficients
  "coefficients".format(self.threshold)


### Learn lifted bilinear state space model with bEDMD

To enable learning of a bilinear model, we modify the EDMD algorithm by defining the basis functions as follows

\begin{equation}
\bs{\hat{\phi}}(\mb x, \mb u) = \begin{bmatrix} \bs{\phi}(\mb x) \\ \bs{\phi}(\mb x) u_1\\ \vdots \\ \bs{\phi}(\mb x)u_m
\end{bmatrix}
\end{equation}

then, we solve the linear regression problem $\bs{\dot{\phi}} = \hat{A}\bs{\hat{\phi}}$ to find the $\hat{A}$ that best
fits the data. If the lifting dimension of $\bs{\phi}(x)$ is $n_{lift}$, then we can parse $\hat{A}$ as follows

\begin{equation}
F = \hat{A}_{(:,:n_{lift})}, \qquad G_1 = \hat{A}_{:,n_{lift}:2n_{lift}}, \qquad \dots \qquad G_m  = \hat{A}_{:,m n_{lift}:(m+1)n_{lift}}
\end{equation}

resulting in the learned bilinear model of the form

\begin{equation}
\mb{\dot{z}} = F \mb z + \sum_{i=1}^m G_i\mb z u_i
\end{equation}


In [29]:
poly_features_bedmd = PolynomialFeatures(degree=2)
poly_features_bedmd.fit(np.zeros((1,n)))
n_lift_bedmd = poly_features_bedmd.n_output_features_
core_basis_bedmd = lambda x: poly_features_bedmd.transform(x)
bedmd_basis = BilinearBasis(n, m, n_lift_bedmd, core_basis_bedmd)
bedmd_basis.construct_basis()

zs_bedmd = np.array([core_basis_bedmd(xs[ii,:-1,:]) for ii in range(n_traj)])
zs_dot_bedmd = np.array([differentiate_vec(zs_bedmd[ii,:,:], t_eval[:-1]) for ii in range(n_traj)])  #Numerical differentiate lifted state
zs_bu_bedmd = np.array([bedmd_basis.lift(xs[ii,:-1,:], us[ii,:,:]) for ii in range(n_traj)])
zs_dot_flat_bedmd = zs_dot_bedmd.T.reshape((n_lift_bedmd, n_data_pts),order=order)
zs_bu_flat_bedmd = zs_bu_bedmd.T.reshape(((m+1)*n_lift_bedmd, n_data_pts),order=order)

NameError: name 'PolynomialFeatures' is not defined

In [ ]:
bedmd_model = STLSQ(threshold=threshold_bedmd)
bedmd_standardizer = preprocessing.StandardScaler(with_mean=False)
bedmd_standardizer.fit(zs_bu_flat_bedmd.T)
X_bedmd = bedmd_standardizer.transform(zs_bu_flat_bedmd.T)
y_bedmd = zs_dot_flat_bedmd.T
bedmd_model.fit(X_bedmd, y_bedmd)

# Parse learned coefficients into appropriate dynamics matrices
coefs_bedmd = bedmd_standardizer.transform(bedmd_model.coef_)
F_bedmd = coefs_bedmd[:,:n_lift_bedmd]
G_bedmd = []
for ii in range(m):
    G_bedmd.append(coefs_bedmd[:,n_lift_bedmd*(ii+1):n_lift_bedmd*(ii+2)])

C_h_bedmd = np.zeros((n,n_lift_bedmd))
C_h_bedmd[:,1:n+1] = np.eye(n)
bedmd_sys = BilinearDynamics(n_lift_bedmd, m, F_bedmd, G_bedmd, core_basis_bedmd)

### Compare prediction performance of the linear and bilinear models

In [ ]:
from matplotlib import pyplot as plt
from koopman_core.controllers.openloop_controller import OpenLoopController

n_traj_test = 25
xs_test, xs_edmd_test, xs_bedmd_test, us_test, ts_test = [], [], [], [], []
for ii in range(n_traj_test):
    x_0 = np.asarray([rn.uniform(-i,i)  for i in traj_bounds])
    xs_tmp, us_tmp = finite_dim_koop_sys.simulate(x_0, pert_lqr_controller, t_eval)

    ol_controller = OpenLoopController(bedmd_sys, us_tmp, t_eval[:-1])
    z_0 = basis_edmd(np.atleast_2d(x_0)).squeeze()

    zs_edmd_tmp, _ = edmd_sys.simulate(z_0, ol_controller, t_eval[:-1])
    xs_edmd_tmp = np.dot(C_h_edmd, zs_edmd_tmp.T)

    z_0 = core_basis_bedmd(np.atleast_2d(x_0)).squeeze()
    zs_bedmd_tmp, _ = bedmd_sys.simulate(z_0, ol_controller, t_eval[:-1])
    xs_bedmd_tmp = np.dot(C_h_bedmd, zs_bedmd_tmp.T)

    xs_test.append(xs_tmp.T)
    xs_edmd_test.append(xs_edmd_tmp)
    xs_bedmd_test.append(xs_bedmd_tmp)
    us_test.append(us_tmp)
    ts_test.append(t_eval)

xs_test, xs_edmd_test, xs_bedmd_test, us_test, ts_test = np.array(xs_test), np.array(xs_edmd_test), np.array(xs_bedmd_test), np.array(us_test), np.array(ts_test)

error_edmd = xs_test[:,:,:-1] - xs_edmd_test
error_edmd_mean = np.mean(error_edmd, axis=0)
error_edmd_std = np.std(error_edmd, axis=0)
mse_edmd = np.mean(np.mean(np.mean(np.square(error_edmd))))

error_bedmd = xs_test[:,:,:-1] - xs_bedmd_test
error_bedmd_mean = np.mean(error_bedmd, axis=0)
error_bedmd_std = np.std(error_bedmd, axis=0)
mse_bedmd = np.mean(np.mean(np.mean(np.square(error_bedmd))))

fig, axs = plt.subplots(2, 2, figsize=(10, 6))
ylabels = ['$e_{q_1}$', '$e_{q_2}$', '$e_{\dot{q}_1}$', '$e_{\dot{q}_2}$']
fig.suptitle('Open loop predicition error of EDMD and bilinear EDMD models', y=1.025, fontsize=18)
for ax, err_edmd_mean, err_edmd_std, err_bedmd_mean, err_bedmd_std, ylabel in zip(axs.flatten(), error_edmd_mean, error_edmd_std, error_bedmd_mean, error_bedmd_std, ylabels):
    ax.plot(ts_test[0,:-1], err_edmd_mean, linewidth=3, label='mean, EDMD')
    ax.fill_between(ts_test[0,:-1], err_edmd_mean-err_edmd_std, err_edmd_mean+err_edmd_std, alpha=0.2, label='std, EDMD')
    ax.plot(ts_test[0,:-1], err_bedmd_mean, linewidth=3, label='mean, bEDMD')
    ax.fill_between(ts_test[0,:-1], err_bedmd_mean-err_bedmd_std, err_bedmd_mean+err_bedmd_std, alpha=0.2, label='std, bEDMD')
    ax.set_ylabel(ylabel, fontsize=16)
    ax.grid()
    ax.set_xlabel('$t$ (sec)', fontsize=16)

ax.legend()
plt.tight_layout()
plt.show()
print('Lifting dimension EDMD: ', n_lift_edmd, 'Lifting dimension bEDMD: ', n_lift_bedmd*(m+1))
print('MSE EDMD: ', mse_edmd, '\nMSE bilinear EDMD: ', mse_bedmd, '\nImprovement: ', (1-mse_bedmd/mse_edmd)*100, ' percent')

